In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import math
import random
import itertools

In [7]:
class City:
    def __init__(self,no_of_loc,locations=None):
    #here locations is a dictionary,{ nodeid: (x, y)} for specific locations
        self.G = nx.Graph()
        self.pos = {}
        self.num_nodes = no_of_loc
        if locations:
            self.pos = {k: np.array(v) for k, v in locations.items()}
            for i, coord in self.pos.items():
                self.G.add_node(i, pos=coord)
        else:
            self.pos = {i: np.array([random.uniform(0, 30), random.uniform(0, 30)]) for i in range(1, num_nodes+1)}
            for i in range(1, num_nodes+1):
                self.G.add_node(i, pos=self.pos[i])
        self._build_roads()
        self.edges = list(self.G.edges())
        self.edge_map = {tuple(sorted(e)): i for i, e in enumerate(self.edges)}
        self.num_edges = len(self.edges)
        self.true_alphas = np.random.uniform(-1.0, 1.0, self.num_edges)
        self.learned_alphas = np.zeros(self.num_edges)

        
    def _build_roads(self):
        nodes = list(self.G.nodes())
        threshold1 = 10.0 
        threshold2 = 25.0

        for i in range(len(nodes)):
            for j in range(i + 1, len(nodes)):
                u, v = nodes[i], nodes[j]
                d = float(np.linalg.norm(self.pos[u] - self.pos[v]))
                if d <= threshold1 or d>=threshold2:
                    self.G.add_edge(u, v, distance=d, weight=d)
                    
        if not nx.is_connected(self.G):
            components = list(nx.connected_components(G))
            print(components)
            for k in range(len(components)-1):
                u = random.choice(list(components[k]))
                v = random.choice(list(components[k+1]))
                d = float(np.linalg.norm(pos[a] - pos[b]))
                G.add_edge(u, v, distance=d)
                
    def get_distance_matrix(self,use_traffic=False):
        nodes = sorted(list(self.G.nodes()))
        n = len(nodes)
        matrix = np.full((n, n), np.inf)
        np.fill_diagonal(matrix, 0.0)
        for u, v, data in self.G.edges(data=True):
            if use_traffic:
                idx = self.edge_map.get(tuple(sorted((u, v))))
                alpha = self.learned_alphas[idx]
                cost = data['distance'] / (SPEED_BASE * np.exp(alpha))
                data['weights']=cost
            else:
                # Just Physical Distance
                cost = data['distance']
            matrix[u][v] = cost
            matrix[u][v] = cost
        return matrix, nodes
    def get_shortest_path_matrix(self, use_traffic=False):
        for u, v, data in self.G.edges(data=True):
            if use_traffic:
                idx = self.edge_map.get(tuple(sorted((u, v))))
                alpha = self.learned_alphas[idx]
                cost = data['distance'] / (SPEED_BASE * np.exp(alpha))
                data['weights']=cost
                str='weights'
            else:
                # Just Physical Distance
                cost = data['distance']
                str='distance'
        path_gen = nx.all_pairs_dijkstra_path_length(self.G, weight=str)
        n = self.num_nodes
        matrix = np.zeros((n, n))
        for src, targets in path_gen:
            for dst, val in targets.items():
                matrix[src][dst] = val
        
        return matrix
    def _get_edge_true_time(self, u, v):
        idx = self.edge_map.get(tuple(sorted((u, v))))
        if idx is None: return 0
        return self.G[u][v]['distance'] / (SPEED_BASE * np.exp(self.true_alphas[idx]))
        
    def Tobs(self,route):
        total_time = 0.0
        for i in range(len(route) - 1):
            u, v = route[i], route[i+1]
            if self.G.has_edge(u, v):
                total_time += self._get_edge_true_time(u, v)
        return total_time
    def assign_orders_kmeans(self, agent_starts, order_nodes,iterations=20):
        distanceMatrix = self.get_shortest_path_matrix(use_traffic=False)
        # agent_starts contains nodes
        # self.pos[node] contains coordinates
        # initially centroids are coordinates of agent_starts
        centroids = agents[:]
        clusters = [[] for _ in range(k)]
        for _ in range(iterations):
            clusters = [[] for _ in range(k)]
            
            for order in order_nodes:
                best_idx = -1
                min_dist = float("inf")
                
                for i in range(k):
                    depot_node = centroids[i]
                    dist = distanceMatrix[depot_node][order]

                    if dist < min_dist:
                        min_dist = dist
                        best_idx = i
                clusters[best_idx].append(order)
            for i in range(k):
                cluster_orders = clusters[i]

                if not cluster_orders:
                    # Keep original depot node if cluster is empty
                    new_centroids.append(agents[i])
                    continue

                # RECOMPUTE centroid as the order closest to all others in that cluster
                best_node = None
                best_sum_dist = float("inf")

                for candidate in cluster_orders:
                    # Sum of distances from candidate to all other nodes in cluster
                    total = sum(distanceMatrix[candidate][o] for o in cluster_orders)
                    
                    if total < best_sum_dist:
                        best_sum_dist = total
                        best_node = candidate

                new_centroids.append(best_node)

            centroids = new_centroids

        # Create final assignment dictionary
        assignments = {}
        for i in range(k):
            assignments[agents[i]] = clusters[i]
            
        return assignments
    def solve_tsp_dp():
        
        return None

    def route(self,start,orders,use_traffic=False):
        shortestpath= self.get_shortest_path_matrix(use_traffic)
        distance_matrix=self.get_distance_matrix(use_traffic)
        if not orders:
            return [start]
        if len(orders) == 1:
            return shortestpath[start][orders[0]]
        else:
            return self.solve_tsp_dp(distance_matrix,start, orders, use_learned_traffic)
    def visualize_graph(self):
        plt.figure(figsize=(30, 30))
        nx.draw_networkx_nodes(self.G, self.pos, node_size=150, node_color="skyblue")
        nx.draw_networkx_edges(self.G, self.pos, width=1.5)

        nx.draw_networkx_labels(self.G, self.pos, font_size=12)
        edge_labels = {
            (u, v): f"{G[u][v]['distance']:.1f}"
            for u, v in G.edges()
        }
        nx.draw_networkx_edge_labels(self.G,self.pos, edge_labels=edge_labels, font_size=9)
        if title:
            plt.title(title)
        plt.axis("equal")
        plt.show()
        draw_graph(self.G, self.pos, title="Graph Visualization")

    def visualize_route(self, route, title="Delivery Route"):

        plt.figure(figsize=(10, 8))
        

        nx.draw(self.G, self.pos, 
                node_color='lightgray', node_size=50, 
                edge_color='lightgray', width=1, 
                with_labels=True, font_size=8)
        

        path_edges = []
        for i in range(len(route) - 1):
            u, v = route[i], route[i+1]
            if self.G.has_edge(u, v):
                path_edges.append((u, v))
            else:
                sub_path = nx.shortest_path(self.G, u, v, weight='distance')
                for k in range(len(sub_path)-1):
                    path_edges.append((sub_path[k], sub_path[k+1]))
        
        nx.draw_networkx_edges(self.G, self.pos, edgelist=path_edges, 
                               edge_color='blue', width=3)
        
        nx.draw_networkx_nodes(self.G, self.pos, nodelist=[route[0]], node_color='green', node_size=150, label="Start")
        nx.draw_networkx_nodes(self.G, self.pos, nodelist=[route[-1]], node_color='red', node_size=150, label="End")
        
        plt.title(f"{title}")
        plt.legend()
        plt.show()
        
    